In [3]:
import os
import argparse
import io
import glob
import hashlib
from pathlib import Path
from PIL import Image
import subprocess
import cv2

import tensorflow as tf
from object_detection.utils import dataset_util


In [4]:
DATASET_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset"
TF_FILE_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/tf_data.tfrecord"

In [5]:
class_names = {0:"door",
              1:"handle",
              2:"cabinet door",
              3:"refrigerator door"}

In [6]:
def create_tf_example(img_path:str,
                      annotation_path:str,
                     class_names:dict):
    
    # ------------------------------------------------------------
    # Read the image
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_img = fid.read()

    encoded_img_io = io.BytesIO(encoded_img)
    image = Image.open(encoded_img_io)
    img_width, img_height = image.size
    
#     key = hashlib.sha256(encoded_img).hexdigest()
    
    fn = Path(img_path).stem
    
    # ------------------------------------------------------------
    # Read the annotation
    x_1,y_1,x_2,y_2 = [],[],[],[]
    classes_text, clss_list = [],[]
    with open(annotation_path, 'r') as li:
        annotations = li.readlines()
        for i in annotations:
#             print(list(map(float,i.strip().split(' '))))
            label,x,y,w,h = list(map(float,i.strip().split(' ')))
            x_1.append(float(x - (w / 2)))
            y_1.append(float(y - (h / 2)))
            x_2.append(float(x + (w / 2)))
            y_2.append(float(y + (h / 2)))
            clss_list.append(int(label)+1)
            classes_text.append(class_names[int(label)].encode('utf8'))
    
    # --------------------------------------------------------------
    # TODO(user): Populate the following variables from your example.
    width,height  = image.size    # Image width , Image height
    filename = fn.encode('utf8') # Filename of the image. Empty if image is not from file
    encoded_image_data = encoded_img # Encoded image bytes
    image_format = b'png' # b'jpeg' or b'png'

    xmins = x_1 # List of normalized left x coordinates in bounding box (1 per box)
    ymins = y_1 # List of normalized top y coordinates in bounding box (1 per box)
    xmaxs = x_2 # List of normalized right x coordinates in bounding box (1 per box)
    ymaxs = y_2 # List of normalized bottom y coordinates in bounding box (1 per box)

    #classes_text = [k.encode('utf8') for k in class_names.values()] # List of string class name of bounding box (1 per box)
    classes = clss_list # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
      }))
    
    return tf_example

In [7]:
writer =  tf.io.TFRecordWriter(TF_FILE_PATH)

img_file_list = glob.glob(os.path.join(DATASET_PATH,"*.png"), recursive=True)

for i,img in enumerate(img_file_list) :
    name = Path(img).stem

    img_path = os.path.join(DATASET_PATH , name + ".png")
    annotation_path = os.path.join(DATASET_PATH ,name  + ".txt")
    
    print(img_path)
    
    tf_record = create_tf_example(img_path,annotation_path,class_names)
    writer.write(tf_record.SerializeToString())
    
    print(i," ",name," written to tf record.")

writer.close()


/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/c279e14b172412aa.png
0   c279e14b172412aa  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/5f9cbca55c53abfb.png
1   5f9cbca55c53abfb  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/bb561fb5cebe8415.png
2   bb561fb5cebe8415  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/93b5e3ab86711a0b.png
3   93b5e3ab86711a0b  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/0e872e10f2531364.png
4   0e872e10f2531364  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/b0b08bdfbb3612db.png
5   b0b08bdfbb3612db  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/3d94428eaf8808c2.png
6   3d94428eaf8808c2  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/1adffb670386a3fd.png
7   1adffb670386a3fd  written to tf record.
/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset/0cc